In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import pickle 
from itertools import chain
from datetime import datetime
from collections import defaultdict

from typing import List, Dict, Optional, Iterable, Tuple

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from datasets import load_dataset
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

import torch
from torch import nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

import tokenizers
from tokenizers import Tokenizer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.processors import TemplateProcessing

/home/alm/miniforge3/envs/assistant_nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [131]:
from stat_lm import Tokenizer, StatLM, GenerationConfig

In [4]:
# ds_name_2 = 'IlyaGusev/stihi_ru'

def get_dataset(train_size: int,
                test_size: int,
                ds_name_1: str = 'IlyaGusev/gazeta',
               ): 
    
    train_dataset = load_dataset(ds_name_1, split='train')
    test_dataset = load_dataset(ds_name_1, split='test')

    train_df = pd.DataFrame(train_dataset).iloc[:train_size]
    print(train_df.shape)

    test_df = pd.DataFrame(test_dataset)[:test_size]
    print(test_df.shape)

    train_texts = (train_df['title'] + '\n' + train_df['text']).tolist()
    test_texts = (test_df['title'] + '\n' + test_df['text']).tolist()
    
    return train_texts, test_texts

In [86]:
ds_name_1 = "IlyaGusev/gazeta"
train_size = 60964
train_size = 5000
test_size = 5000

train_dataset = load_dataset(ds_name_1, split='train')
test_dataset = load_dataset(ds_name_1, split='test')

train_df = pd.DataFrame(train_dataset).iloc[:train_size]
print(train_df.shape)

test_df = pd.DataFrame(test_dataset)[:test_size]
print(test_df.shape)

train_texts = (train_df['title'] + '\n' + train_df['text']).tolist()
test_texts = (test_df['title'] + '\n' + test_df['text']).tolist()

(5000, 5)
(5000, 5)


In [87]:
all_texts = train_texts + test_texts

In [88]:
all_texts = test_texts

In [89]:
print(len(all_texts))

5000


In [90]:
tokenizer = Tokenizer().build_vocab(all_texts)

In [91]:
len(tokenizer.vocab)

156197

In [92]:
text_example = "В России люди любят искать приключения на голову"

In [93]:
tokenizer.encode(text_example)

[155768, 6797, 55871, 4855, 49768, 96698, 28478, 124855, 156194]

In [94]:
tokenizer._tokenize(text_example, append_eos_token=False)

['в', 'россии', 'люди', 'любят', 'искать', 'приключения', 'на', 'голову']

In [95]:
tokenizer.decode(tokenizer.encode(text_example), remove_special_tokens=True)

'в россии люди любят искать приключения на голову'

In [134]:
stat_lm = StatLM(tokenizer, context_size=4, alpha=0.01)

In [135]:
# "обучаем" модель - считаем статистики
stat_lm.train(train_texts)

training: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:06<00:00, 730.34it/s]


In [136]:
for text in test_texts[32:37]:
    title = text.split('\n')[0]
    print(title)

Количество импортной продукции в России снизилось на 30% 
Называвший русский язык «убогим» профессор ВШЭ Гусейнов уволился 
Россияне оценили долю бесполезно потраченного на работе времени 
Школьники рассказали о тревожном состоянии после 1 сентября 
В США статую Конфедерации хотят заменить памятником Черной Пантере 


In [144]:
config = {
    'temperature': 1.2,
    'max_tokens': 24,
    'sample_top_p': 0.0005,
    'decoding_strategy': 'top-p',
    'gen_decay': 1e-32,
}

generation_config = GenerationConfig(temperature=config['temperature'],
                                     max_tokens=config['max_tokens'],
                                     sample_top_p=config['sample_top_p'],
                                     decoding_strategy=config['decoding_strategy'],
                                     gen_decay=config['gen_decay'],
                                     remove_special_tokens=True)

In [148]:
for text in test_texts[50:55]:
    title = text.split('\n')[0]
    generated = stat_lm.generate_text(title, generation_config)
    print(generated['total_text'])
    print(generated['finish_reason'], '\n')

за крупнейшим взломом twitter стоит 16 - летний подросток . врачи зафиксировали у него черепно намекало прикосновений благодарен анонсировано веб указывает распространят
max tokens 

ni : российский гермес станет убийцей западных танков вихлянцев подкрепление an
end of text 

автор хита i like to move it умер на фоне обвинений в насилии шары изучила sinoruss подрядчики аппендицита немалахов вспыхнуло аургазинском анонсировано педвузов герек
max tokens 

названа опасность постоянно включенного bluetooth баширова непредумышленным надобности аппендицита вспыхнуло ютуберов немалахов давосского отторжения ландшафты перекрыта метеорологическая отменили сибирских кутаиси месседжеров колесом заливаемости transportation
max tokens 

лавров сообщил о скорой встрече лукашенко с путиным в москве аппендицита smoke синодальный банкротные алтуфьеве кутаиси подкрепление масштабы проваливается легионы азамат отменили тори монополистами
max tokens 



In [146]:
for text in train_texts[21:25]:
    title = text.split('\n')[0]
    generated = stat_lm.generate_text(title, generation_config)
    print(generated['total_text'])
    print(generated['finish_reason'], '\n')

автобус тормозит в развитии во вторник столичное правительство обсудило итоги реализации городской целевой программы развития наземного пассажирского транспорта на прошедшие три года не исключено
max tokens 

реформы начнутся , когда деньги авторы и исполнители стратегии - 2010 шары давосского приговорено
end of text 

зарплата превыше всего сегодня президиум высшего арбитражного суда вас рф поставил точку в матче , однако его удар оказался заблокирован . реальную возможность
max tokens 

лена борется до конца во вторник на пресс - конференции в четверг . как сообщает , руководство островитян готово сделать россиянину весьма заманчивое
max tokens 



In [156]:
tokenizer.save('tokenizer_alm.pkl')

True

In [157]:
stat_lm.save_stat('stat_lm_alm.pkl')

True